In [2]:
from loss import *
import torch
from models import *
from src.dataload import *
from tqdm.notebook import tqdm
from metrics.metrics import *


In [3]:

#dataloader UIEBUIEB
train_loader_UIEB, test_loader_UIEB = create_dataloader(dataset_name="UIEB", dataset_path="data")

#dataloader TURBID
train_loader_TURBID, test_loader_TURBID = create_dataloader(dataset_name="TURBID", dataset_path="data")

#dataloader LSUI
train_loader_LSUI, test_loader_LSUI = create_dataloader(dataset_name="LSUI", dataset_path="data")

Train raw: 712 Train ref: 712 
Test raw: 178 Test ref: 178



ValueError: Default process group has not been initialized, please make sure to call init_process_group.

In [ ]:
def competicao():
    ##importa dataset

    ##importa loss

    ##importa modelos

    ##importar metricas

    ##inicia estrutura com ddp e torchrun

    ##treinar por 100 epocas para cada modelo//total 1500 epocas pq sao 15 loss

    ##salvar checkpoints

    ##roda as metricas

    

    modelos = [Unet_model(), Vit_model(),VAE_model()]
    #modelos = [VAE_model()]

    return modelos



In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.randn(1, 3, 256, 256).to(device)
modelos = load_models()
for model in modelos:
    model = model.to(device)
    if type(model(x)) is tuple:
        print(model(x)[0].shape, model(x)[1].shape, model(x)[2].shape)
    else:
        print(model(x).shape)

if type(model(x)) is tuple:
        print(model(x)[0].shape, model(x)[1].shape, model(x)[2].shape)
    else:
        print(model(x).shape)

torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 16]) torch.Size([1, 16])


### Notas sobre as Loss
A loss lch nao esta funcionando.

Serao reajustadas para o intervalo de zero a 1 :
* Histogram
* darkchannel
* lch nao funciona
* hsv
### Notas

* Maiores problemas no autoencoder e em seu funcionamento. A dimensao latente de 16 nao gera nada aparentemente. 
* Talvez alterar o intervalo numero dos canais de cor ajude neste caso. O intervalo entre zero e 1 fazer 1/loss -1
* Terei de retreinar para passar os resultados
* Para selecionar as melhores loss function devemos utilizar outro metodo. Nao sera possivel utilizar as metricas devido a grande quantidade de funcoes que nao gera imagem como a MSE e a SSIM. Essas funcoes tem de ser investigadas. 
* Como o tempo esta curso talvez seja melhor selecionar visualmente as melhores funcoes e agrupar de acordo com as necessidades do script.
* Ainda preciso revisar os excperimentos do sbr


In [13]:
#modelos = load_models()
import torch
from loss import *
from models import *
from src.dataload import *
from tqdm.notebook import tqdm
from metrics.metrics import *
loss_battle = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#loss_battle.extend(build_perceptual_losses(rank=device))
loss_battle.extend(build_channel_losses(rank = device))
#loss_battle.extend(build_structural_losses(rank = device))

print(f"{len(loss_battle)} loss functions to train with\n")
#print(f"{len(modelos)} models to train with")
x = torch.randn(1, 3, 256, 256).to(device)
y = torch.randn(1, 3, 256, 256).to(device)
for loss_fn in loss_battle:
    print(f"    {loss_fn.name}, result({loss_fn(x, y)})")

5 loss functions to train with

    HistogramColorLoss, result(6022.24755859375)
    angular_color_loss, result(0.9995093941688538)
    DarkChannelLoss, result(261.767578125)
    LCHChannelLoss, result(nan)
    HSVChannelLoss, result(8594029568.0)


## Outos ajustes ddp

In [ ]:
import os
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler, TensorDataset

def setup(rank, world_size):
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)
    print(f"Rank {rank} initialized.")

def cleanup():
    dist.destroy_process_group()

def train(rank, world_size, epochs=100):
    setup(rank, world_size)

    #dataloader UIEBUIEB
    train_loader_UIEB, test_loader_UIEB,sampler = create_dataloader(dataset_name="UIEB", dataset_path="data",world_size=world_size,rank=rank,rank_test=0)

    model = model.cuda(rank)##carrega o modelo
    ddp_model = DDP(model, device_ids=[rank])

    criterion = nn.MSELoss().cuda(rank)##carrega loss function
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)


    for epoch in range(epochs):
        ddp_model.train()
        sampler.set_epoch(epoch)
        for batch_idx, (data, target) in enumerate(train_loader_UIEB):
            data, target = data.cuda(rank), target.cuda(rank)

            optimizer.zero_grad()
            output = ddp_model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f"Rank {rank}, Epoch [{epoch}/{epochs}], Batch [{batch_idx}/{len(dataloader)}], Loss: {loss.item()}")

    cleanup()

if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", type=int, default=10, help="Number of epochs")
    parser.add_argument("--nodes", type=int, default=1, help="Number of nodes")
    parser.add_argument("--gpus", type=int, default=2, help="Number of GPUs per node")
    args = parser.parse_args()

    world_size = args.nodes * args.gpus

    torch.multiprocessing.spawn(train, args=(world_size, args.epochs), nprocs=args.gpus, join=True)


In [ ]:
%torchrun --nnodes=2 --nproc_per_node=2 --node_rank=0 --master_addr="10.228.252.209" --master_port=12355 train_ddp.py --nodes=2 --gpus=2 --epochs=10
#10.228.247.253

In [ ]:
import os
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler, TensorDataset
from loss import *
import torch
from models import *
from src.dataload import *
from tqdm.notebook import tqdm
from metrics.metrics import *

def setup(rank, world_size):
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)
    print(f"Rank {rank} initialized.")

def cleanup():
    dist.destroy_process_group()

class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(10, 1)

    def forward(self, x):
        return self.linear(x)

def train_one_model(rank, world_size, epochs, loss_fn, model_name, model):
    setup(rank, world_size)

    #dataloader UIEBUIEB
    train_loader_UIEB, test_loader_UIEB,sampler = create_dataloader(dataset_name="UIEB", dataset_path="data",world_size=world_size,rank=rank,rank_test=0)

    modell = model.cuda(rank)
    ddp_model = DDP(modell, device_ids=[rank])

    criterion = loss_fn().cuda(rank)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)

    for epoch in range(epochs):
        ddp_model.train()
        sampler.set_epoch(epoch)
        for batch_idx, (data, target) in enumerate(train_loader_UIEB):
            data, target = data.cuda(rank), target.cuda(rank)

            optimizer.zero_grad()
            output = ddp_model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0:
                print(f"Rank {rank}, Epoch [{epoch}/{epochs}], Batch [{batch_idx}/{len(train_loader_UIEB)}], Loss: {loss.item()}")
    ##Salve Dir para salvar os checkpoints
    ckpt_savedir='output/ckpt_battle/'
    if not os.path.exists(ckpt_savedir):
        os.makedirs(ckpt_savedir)
    if rank == 0:
        # Salvar o estado do modelo original, não o DDP
        torch.save(model.state_dict(), f"{ckpt_savedir}{model_name}_ckpt.pth")
        psnr_list, ssim_list, uciqe_list, uiqm_list = [], [], [], []
        # Avaliar o modelo
        model.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_loader_UIEB):
                data, target = data.cuda(rank), target.cuda(rank)
                #calcula a metrica
                targets = target.cpu().numpy()
                predictions = model(data.cuda(rank)).cpu().numpy()
                psnr_value, ssim_value, uciqe_, uiqm = calculate_metrics(predictions, targets)
                psnr_list.append(psnr_value)
                ssim_list.append(ssim_value)
                uciqe_list.append(uciqe_)
                uiqm_list.append(uiqm)
        avg_ssim = sum(ssim_list) / len(ssim_list)
        avg_psnr = sum(psnr_list) / len(psnr_list)
        avg_uciqe = sum(uciqe_list) / len(uciqe_list)
        avg_uiqm = sum(uiqm_list) / len(uiqm_list)
           
        # Salvar métricas em um arquivo
        results_savedir='output/results_battle/'
        if not os.path.exists(results_savedir):
            os.makedirs(results_savedir)
        
        with open(f'output/{model_name}_metrics.txt', 'w') as f:
            f.write(f"""avg_ssim:{avg_ssim}\navg_psnr:{avg_psnr}\navg_uciqe:{avg_uciqe}\navg_uiqm:{avg_uiqm}""")
            print(f"Metrics for {model_name} saved to {results_savedir}/{model_name}_metrics.txt")


    cleanup()

def train(rank, world_size, epochs):
    
    modelos = load_models()
    loss_battle = []

    loss_battle.extend(build_perceptual_losses())
    loss_battle.extend(build_channel_losses())
    loss_battle.extend(build_structural_losses())

    for model in modelos:
        for loss_fn in loss_battle:
            model_name = model.__class__.__name__ + "_" + loss_fn.__class__.__name__
            train_one_model(rank, world_size, epochs, loss_fn, model_name, model)        
        
if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", type=int, default=100, help="Number of epochs per model")
    parser.add_argument("--nodes", type=int, default=1, help="Number of nodes")
    parser.add_argument("--gpus", type=int, default=2, help="Number of GPUs per node")
    args = parser.parse_args()

    world_size = args.nodes * args.gpus

    torch.multiprocessing.spawn(train, args=(world_size, args.epochs), nprocs=args.gpus, join=True)


# Rodar no cluster

In [ ]:
torchrun --nnodes=2 --nproc_per_node=2 --node_rank=0 --master_addr="10.228.252.209" --master_port=22 main.py --nodes=2 --gpus=2 --epochs=100


In [ ]:
#local
torchrun --nproc_per_node=2 main.py --nodes=1 --gpus=2 --epochs=100
